<a href="https://colab.research.google.com/github/blue-create/langlens/blob/main/format_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# access data 
%cd /content/drive/MyDrive/data/

/content/drive/MyDrive/data


In [ ]:
# check number of files 
import os 

num_files = len(os.listdir("."))
print("Number of files in the folder: ", num_files)

Number of files in the folder:  2


In [ ]:
import pandas as pd

dataset = pd.DataFrame(columns=['Filename', 'Type', 'Content'])

for entry in sorted(os.listdir('.')):
    entry_path = os.path.join('.', entry)
    if os.path.isdir(entry_path):
        # Entry is a folder, add XML files in the folder to the dataset
        for file in sorted(os.listdir(entry_path)):
            if file.endswith('.xml'):
                file_path = os.path.join(entry_path, file)
                with open(file_path, 'r') as f:
                    content = f.read()
                dataset = dataset.append({'Filename': file, 'Type': 'XML', 'Content': content}, ignore_index=True)
    elif os.path.isfile(entry_path) and entry.endswith('.xml'):
        # Entry is an XML file, add it to the dataset
        with open(entry_path, 'r') as f:
            content = f.read()
        dataset = dataset.append({'Filename': entry, 'Type': 'XML', 'Content': content}, ignore_index=True)

print(dataset.head())


Empty DataFrame
Columns: [Filename, Type, Content]
Index: []


In [ ]:
# unzip all files

import zipfile
import shutil

# Path to the raw folder
raw_folder = "./Raw"

# Path to the unzipped folder
unzipped_folder = "./unzipped"

# Loop through all the files in the raw folder
for filename in os.listdir(raw_folder):
    if filename.endswith(".zip"):
        # Check if the file is a valid zip file
        if zipfile.is_zipfile(os.path.join(raw_folder, filename)):
            # Create a ZipFile object for the current zip file
            with zipfile.ZipFile(os.path.join(raw_folder, filename), "r") as zip_ref:
                # Extract all the contents of the zip file to the unzipped folder
                zip_ref.extractall(unzipped_folder)
        else:
            print(f"File {filename} is not a valid zip file and will not be extracted.")

In [ ]:
# Loop through all the files in the Raw folder
for filename in os.listdir(raw_folder):
    if filename.endswith(".zip"):
        # Get the first part of the filename before the .zip extension
        name = filename.split(".")[0]

        # Loop through all the files in the unzipped folder
        for xml_filename in os.listdir(unzipped_folder):
            if xml_filename.startswith(name) and xml_filename.endswith(".xml"):
                break
        else:
            # A corresponding XML file doesn't exist
            print(f"No corresponding XML file exists for {filename}")

No corresponding XML file exists for NBPC.zip


In [158]:
import pandas as pd
import xml.etree.ElementTree as ET
from tqdm import tqdm # for progress bar 

data_folder = "unzipped"
prefix = "FAZE_"  # set the prefix you want to test here

# create a list of all xml files in the data folder
xml_files = [f for f in os.listdir(data_folder) if f.endswith('.xml')]

# filter the xml files by prefix
prefix_files = [f for f in xml_files if f.startswith(prefix)]

In [ ]:
# create an empty pandas DataFrame to store the xml data
xml_data = pd.DataFrame(columns=["name",
                                 "jahrgang", 
                                 "datum", 
                                 "ressort", 
                                 #"titel", 
                                 "untertitel", 
                                 "text"
                                 ])

# iterate over the xml files and extract the data
for xml_file in tqdm(prefix_files):
    xml_path = os.path.join(data_folder, xml_file)
    tree = ET.parse(xml_path)
    root = tree.getroot()
    for artikel in root.findall('artikel'):
        name = artikel.find('metadaten/quelle/name').text
        jahrgang = artikel.find('metadaten/quelle/jahrgang').text
        datum = artikel.find('metadaten/quelle/datum').text

        ressort = artikel.find('inhalt/titel-liste/ressort').text
        #titel = artikel.find('inhalt/titel-liste/titel').text # does not work for some reason, needs fixing

        untertitel_elem = artikel.find('inhalt/titel-liste/untertitel')
        untertitel = untertitel_elem.text if untertitel is not None else None

        text = []
        # Find the 'text' element
        text_elem = artikel.find('inhalt/text')
        try: 
            # Extract all the 'p' elements inside the 'text' element
            p_elems = text_elem.findall('p')
            # Loop over the 'p' elements and extract their text content
            for p_elem in p_elems:
                p_text = p_elem.text
                text.append(p_text)
        except: 
            pass 


        row = pd.DataFrame({'name': name, 
               'jahrgang': jahrgang, 
               'datum': datum,
               'ressort':ressort,
               #'titel': titel, 
               'untertitel': untertitel, 
               'text': text})
        xml_data = pd.concat([xml_data, row], ignore_index=True)


  0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
# create the Excel file for the prefix
excel_file = os.path.join("excel", f"{prefix}data.xlsx")
xml_data.to_excel(excel_file, index=False)

things to do: 
- connect to github 
- create and store dataset 
- do some descriptive analysis: number of articles per newspaper, number of newspapers, number of topics per newspaper, etc. 
- do filtering: german newspapers only, DA related topics only 
- topic analysis: run the filtered dataset through a generic topic model 